Projet Big  Data Binomes:
- Wacim BELAHCEL, Formation Alternance
- Imad Oualid KACIMI, Formation Alternance

#1. Mise en place de l'environnement de travail

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install coclust

In [ ]:
!pip install pandavro

In [ ]:
!pip install pyarrow

In [ ]:
!pip install pyngrok

In [ ]:
!mkdir ~/.ngrok2
!echo "web_addr: localhost:5050" > ~/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2’: File exists


In [ ]:
from pyngrok import ngrok

active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
  public_url = tunnel.public_url
  ngrok.disconnect(public_url)

ngrok_tunnel = ngrok.connect(4040, return_ngrok_tunnel=True)
ngrok_tunnel

<NgrokTunnel: "http://8d5f2abc9222.ngrok.io" -> "http://localhost:4040">

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download Spark
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
# unzip it
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
# install findspark 
!pip install -q findspark

In [ ]:
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.0.1 pyspark-shell'
import findspark

findspark.init("spark-3.0.1-bin-hadoop2.7")

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("mon application").setMaster("local[4]")
#spark context
sc = SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# 2. Données :

## 2.a telechargement du jeu de données

In [ ]:
! curl -L "http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz" > 20news-19997.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.5M  100 16.5M    0     0  9617k      0  0:00:01  0:00:01 --:--:-- 9611k


## 2.b decompresser les données:

In [ ]:
!tar xvzf 20news-19997.tar.gz

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
20_newsgroups/soc.religion.christian/20607
20_newsgroups/soc.religion.christian/20608
20_newsgroups/soc.religion.christian/20609
20_newsgroups/soc.religion.christian/20579
20_newsgroups/soc.religion.christian/20580
20_newsgroups/soc.religion.christian/20581
20_newsgroups/soc.religion.christian/20582
20_newsgroups/soc.religion.christian/20583
20_newsgroups/soc.religion.christian/20584
20_newsgroups/soc.religion.christian/20585
20_newsgroups/soc.religion.christian/20586
20_newsgroups/soc.religion.christian/20587
20_newsgroups/soc.religion.christian/20588
20_newsgroups/soc.religion.christian/20589
20_newsgroups/soc.religion.christian/20590
20_newsgroups/soc.religion.christian/20591
20_newsgroups/soc.religion.christian/20592
20_newsgroups/soc.religion.christian/20593
20_newsgroups/soc.religion.christian/20594
20_newsgroups/soc.religion.christian/20595
20_newsgroups/soc.religion.christian/20596
20_newsgroups/soc.r

## 2.c lecture des fichiers :

In [ ]:
rdd_atheism = sc.wholeTextFiles("/content/20_newsgroups/alt.atheism/*")
rdd_baseball = sc.wholeTextFiles("/content/20_newsgroups/rec.sport.baseball/*")


## 2.d splite en entete et corp

In [ ]:
import re

In [ ]:
rdd_atheism_splitted = rdd_atheism.mapValues(lambda x:re.compile("Lines:\s*\d*\n").split(x,1))
rdd_baseball_splitted = rdd_baseball.mapValues(lambda x:re.compile("Lines:\s*\d*\n").split(x,1))

In [ ]:
rdd_atheism_splitted.take(1)

[('file:/content/20_newsgroups/alt.atheism/53517',
  ['Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53517 talk.religion.misc:83888 talk.origins:40989\nNewsgroups: alt.atheism,talk.religion.misc,talk.origins\nPath: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!news.centerline.com!uunet!pipex!uknet!warwick!nott-cs!mips.nott.ac.uk!eczcaw\nFrom: eczcaw@mips.nott.ac.uk (C.Wainwright)\nSubject: Re: After 2000 years, can we say that Christian Morality is \nMessage-ID: <1993Apr16.164117.6949@cs.nott.ac.uk>\nSender: news@cs.nott.ac.uk\nReply-To: eczcaw@mips.nott.ac.uk (C.Wainwright)\nOrganization: Nottingham University\nReferences: <4fm9iYO00iV303voYt@andrew.cmu.edu> <C5Fuo2.FF8@news.cso.uiuc.edu>  <C5L14I.JJ3@news.cso.uiuc.edu>\nDate: Fri, 16 Apr 93 16:41:17 GMT\n',
   '\nIn article <C5L14I.JJ3@news.cso.uiuc.edu>, cobb@alexia.lis.uiuc.edu (Mike Cobb) writes:\n|> In <1993Apr16.010800.11056@scic.intel.com> sbradley@scic.intel.com (Seth J. \n|> Bradley) writes:\n|> \n|> >In article <

## 2.e extraire L'ensemble des entetes
Nous avons décidé d'extraire les entétes aprés avoir fusionner (suite plus bas).

In [ ]:
def header_split(x,cat):
  
  if len(x) == 1:
    return None
    
  entete = x[0]
  past_key=""
  dict_entete = {}
  
  for line in entete.split("\n"):
    res = line.split(': ',1)
    if len(res) != 1:
      dict_entete[res[0]] = res[1]
      past_key = res[0]
    elif past_key != None :
      dict_entete[past_key] += res[0] 
  
  dict_entete["corp_text"] = x[1] 
  dict_entete["type_doc"] = cat
  return dict_entete

In [ ]:
rdd_atheism_final =  rdd_atheism_splitted.mapValues(lambda x: header_split(x,"atheism"))
rdd_baseball_final =  rdd_baseball_splitted.mapValues(lambda x: header_split(x,"baseball"))

In [ ]:
rdd_atheism_final.count()

1000

In [ ]:
rdd_baseball_final.count()

1000

## 2.f Fusion des rdd

In [ ]:
rdd_final = rdd_atheism_final.union(rdd_baseball_final)

In [ ]:
rdd_final.count()

2000

Rendu du rdd_final fusionné, avec l'ensemble des entetes (nous selectionnons nos entete par la suite), et le fichier concerné pour chaque individu<.

In [ ]:
rdd_final.take(1)

[('file:/content/20_newsgroups/alt.atheism/53517',
  {'Date': 'Fri, 16 Apr 93 16:41:17 GMT',
   'From': 'eczcaw@mips.nott.ac.uk (C.Wainwright)',
   'Message-ID': '<1993Apr16.164117.6949@cs.nott.ac.uk>',
   'Newsgroups': 'alt.atheism,talk.religion.misc,talk.origins',
   'Organization': 'Nottingham University',
   'Path': 'cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!news.centerline.com!uunet!pipex!uknet!warwick!nott-cs!mips.nott.ac.uk!eczcaw',
   'References': '<4fm9iYO00iV303voYt@andrew.cmu.edu> <C5Fuo2.FF8@news.cso.uiuc.edu>  <C5L14I.JJ3@news.cso.uiuc.edu>',
   'Reply-To': 'eczcaw@mips.nott.ac.uk (C.Wainwright)',
   'Sender': 'news@cs.nott.ac.uk',
   'Subject': 'Re: After 2000 years, can we say that Christian Morality is ',
   'Xref': 'cantaloupe.srv.cs.cmu.edu alt.atheism:53517 talk.religion.misc:83888 talk.origins:40989',
   'corp_text': '\nIn article <C5L14I.JJ3@news.cso.uiuc.edu>, cobb@alexia.lis.uiuc.edu (Mike Cobb) writes:\n|> In <1993Apr16.010800.11056@scic.intel

Suppressions des individu ne contenant aucun entete.

In [ ]:
rdd_final = rdd_final.filter(lambda x: x[1] != None)

## 2.e Extractions des entetes que nous souhaitons conserver.

fonctions utilitaire pour choix des entetes à conserver.

In [ ]:

def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list


def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3

def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2)

extraction des clés des entétes.

In [ ]:
rdd_keys = rdd_final.map(lambda x: list(x[1].keys()))

Extraction de l'ensemble des Entétes existantes.

In [ ]:
shared_keys_union = rdd_keys.reduce(lambda x,y : Union(x,y))
shared_keys_union

['X-Disclaimer',
 'Followup-To',
 'Article-I.D.',
 'NNTP-Posting-Host',
 'Nntp-Posting-User',
 ' References',
 'X-News-Reader',
 'X-XXDate',
 'References',
 'Organization',
 'Nntp-Posting-Host',
 'Message-ID',
 'X-Mailer',
 'To',
 'type_doc',
 'X-XXMessage-ID',
 'News-Software',
 'Summary',
 'Xref',
 'Sender',
 'Expires',
 'Distribution',
 'Originator',
 'Subject',
 'In-Reply-To',
 'corp_text',
 'X-Newsreader',
 'Supersedes',
 'Date',
 'From',
 'Disclaimer',
 'Approved',
 'In-reply-to',
 'X-UserAgent',
 'X-Sender',
 'Reply-To',
 'Path',
 'Nntp-Posting-Host-[nntpd-681]',
 'X-Posted-From',
 'Newsgroups',
 'Keywords']

Extractions des intersections (Entétes qui se trouvent dans l'ensemble des fichiers), nous avons decidé de conserver les intersection et non pas les unions en ajoutant "Organization, summary, keywords" que nous estimons étre des entétes interessantes.

In [ ]:
shared_keys_intersection = rdd_keys.reduce(lambda x,y : Intersection(x,y))
shared_keys_intersection = list(shared_keys_intersection)
shared_keys_intersection += ["Organization", "Summary","Keywords"]

In [ ]:
shared_keys_intersection

['Subject',
 'Path',
 'corp_text',
 'From',
 'Newsgroups',
 'type_doc',
 'Organization',
 'Summary',
 'Keywords']

In [ ]:
import numpy as np
import os
#ajouter les clés manquante aux individu et les set à None
def add_keys_header(x,shared_keys):
  
  keys_to_add = np.setdiff1d(shared_keys,list(x[1].keys()))

  for keys in keys_to_add:
    x[1][keys] = None
  x[1]["id"] =os.path.basename(x[0])
  return x[1]

#ne garder que les clés qui se trouves dans "shared keys"
def remove_unshared_keys(x,shared_keys):
  
  d = {}
  
  for keys in shared_keys:
    d[keys]= x[keys]
  d["id"]=  x["id"]
  return d





Selection des entétes

In [ ]:
rdd_final = rdd_final.map(lambda x: add_keys_header(x,shared_keys_intersection)).map(lambda x: remove_unshared_keys(x,shared_keys_intersection))

In [ ]:
rdd_final.take(3)

[{'From': 'eczcaw@mips.nott.ac.uk (C.Wainwright)',
  'Keywords': None,
  'Newsgroups': 'alt.atheism,talk.religion.misc,talk.origins',
  'Organization': 'Nottingham University',
  'Path': 'cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!news.centerline.com!uunet!pipex!uknet!warwick!nott-cs!mips.nott.ac.uk!eczcaw',
  'Subject': 'Re: After 2000 years, can we say that Christian Morality is ',
  'Summary': None,
  'corp_text': '\nIn article <C5L14I.JJ3@news.cso.uiuc.edu>, cobb@alexia.lis.uiuc.edu (Mike Cobb) writes:\n|> In <1993Apr16.010800.11056@scic.intel.com> sbradley@scic.intel.com (Seth J. \n|> Bradley) writes:\n|> \n|> >In article <C5JrDE.M4z@news.cso.uiuc.edu> cobb@alexia.lis.uiuc.edu (Mike \n|> Cobb) writes:\n|> >>Theory of Creationism: MY theistic view of the theory of creationism, (there \n|> >>are many others) is stated in Genesis 1.  In the beginning God created\n|> >>the heavens and the earth.\n|> \n|> >This is a belief, not a theory.  A theory makes predictions and

## 2.g transformer le rdd pour que chaque element soit de type pyspark.sql.Row

In [ ]:
from pyspark.sql import Row
df = rdd_final.map(lambda x: Row(**x)).toDF()



## 2.h créer un objet de type dataframe à partir du rdd

In [ ]:
df = rdd_final.map(lambda x: Row(**x)).toDF()

In [ ]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+-------+--------------------+-----+
|             Subject|                Path|           corp_text|                From|          Newsgroups|type_doc|        Organization|Summary|            Keywords|   id|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+-------+--------------------+-----+
|Re: After 2000 ye...|cantaloupe.srv.cs...|
In article <C5L1...|eczcaw@mips.nott....|alt.atheism,talk....| atheism|Nottingham Univer...|   null|                null|53517|
|Re: free moral ag...|cantaloupe.srv.cs...|Distribution: na
...|kmr4@po.CWRU.edu ...|         alt.atheism| atheism|Case Western Rese...|   null|                null|54221|
|Re: After 2000 ye...|cantaloupe.srv.cs...|Message-ID: <kmr4...|kmr4@po.CWRU.edu ...|talk.abortion,alt...| atheism|Case Western Rese...|   n

## save in avro parquet :

### 2.i Avro

In [ ]:
import pandavro as pdx
import pandas as pd
import numpy as np

avro_filename = "df.avro"
pdx.to_avro(avro_filename, df.toPandas())

In [ ]:
saved = pdx.read_avro(avro_filename)
print(saved)

                                                Subject  ...      id
0     Re: After 2000 years, can we say that Christia...  ...   53517
1                                 Re: free moral agency  ...   54221
2     Re: After 2000 years, can we say that Christia...  ...   53089
3                    Re: Amusing atheists and agnostics  ...   53257
4                Re: Yet more Rushdie [Re: ISLAMIC LAW]  ...   53758
...                                                 ...  ...     ...
1989                                           Moe Berg  ...  104699
1990                         PHILS, NL EAST NOT SO WEAK  ...  104570
1991  Re: Strike zone width (was Re: Jose Canseco's ...  ...  105299
1992                           Re: Early BBDDD Returns?  ...  104449
1993                                       Re: Mel Hall  ...  104424

[1994 rows x 10 columns]


### 2.j Parquet

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

parquet_filename = "df.parquet"
table = pa.Table.from_pandas(df.toPandas())
pq.write_table(table, parquet_filename)

In [ ]:
table2 = pq.read_table(parquet_filename)
table2.to_pandas()

,Subject,Path,corp_text,From,Newsgroups,type_doc,Organization,Summary,Keywords,id
0,"Re: After 2000 years, can we say that Christia...",cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu...,"\nIn article <C5L14I.JJ3@news.cso.uiuc.edu>, c...",eczcaw@mips.nott.ac.uk (C.Wainwright),"alt.atheism,talk.religion.misc,talk.origins",atheism,Nottingham University,None,None,53517
1,Re: free moral agency,cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu...,Distribution: na\nMessage-ID: <kmr4.1679.73552...,kmr4@po.CWRU.edu (Keith M. Ryan),alt.atheism,atheism,Case Western Reserve University,None,None,54221
2,"Re: After 2000 years, can we say that Christia...",cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu...,Message-ID: <kmr4.1587.734911207@po.CWRU.edu>\...,kmr4@po.CWRU.edu (Keith M. Ryan),"talk.abortion,alt.atheism,talk.religion.misc",atheism,Case Western Reserve University,None,None,53089
3,Re: Amusing atheists and agnostics,cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cm...,Distribution: world\nMessage-ID: <1r10jcINNt1g...,cfaehl@vesta.unm.edu (Chris Faehl),alt.atheism,atheism,"University of New Mexico, Albuquerque",None,None,53257
4,Re: Yet more Rushdie [Re: ISLAMIC LAW],cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu...,\nIn article <1993Apr15.215833.15970@bnr.ca> (...,jaeger@buphy.bu.edu (Gregg Jaeger),alt.atheism,atheism,Boston University Physics Department,None,None,53758
...,...,...,...,...,...,...,...,...,...,...
1989,Moe Berg,cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cm...,\nNPR this morning had an interview with Linda...,barring@cs.washington.edu (David Barrington),rec.sport.baseball,baseball,"Computer Science & Engineering, U. of Washingt...",None,None,104699
1990,"PHILS, NL EAST NOT SO WEAK",cantaloupe.srv.cs.cmu.edu!rochester!udel!gatec...,"\nI Love it how all of these people are ""blami...",apanjabi@guvax.acc.georgetown.edu,rec.sport.baseball,baseball,Georgetown University,None,None,104570
1991,Re: Strike zone width (was Re: Jose Canseco's ...,cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cm...,\nIn article <2685.2bd51686@atlas.nafb.trw.com...,rstimets@silver.ucs.indiana.edu (robert and st...,rec.sport.baseball,baseball,Indiana University,None,None,105299
1992,Re: Early BBDDD Returns?,cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu...,\nIn article <1993Apr16.073051.9160@news.cs.br...,vince@sscl.uwo.ca,rec.sport.baseball,baseball,Social Science Computing Laboratory,None,None,104449


comme on peut le voir, le fichier parquet est 4 fois moins lourd que le fichier avro.

In [ ]:
!ls -lh

total 233M
-rw-r--r--  1 root  root   17M Feb 19 22:06 20news-19997.tar.gz
drwxr-xr-x 22 28757 staff 4.0K Apr  3  1999 20_newsgroups
-rw-r--r--  1 root  root  3.8M Feb 19 22:33 df.avro
-rw-r--r--  1 root  root  2.2M Feb 19 22:33 df.parquet
drwx------  5 root  root  4.0K Feb 19 19:52 drive
drwxr-xr-x  1 root  root  4.0K Feb 16 16:35 sample_data
-rw-r--r--  1 root  root   120 Feb 19 21:28 sample_kmeans_data.txt
-rw-r--r--  1 root  root  103K Feb 19 21:52 sample_libsvm_data.txt
drwxr-xr-x 13  1000  1000 4.0K Aug 28 08:10 spark-3.0.1-bin-hadoop2.7
-rw-r--r--  1 root  root  210M Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz
drwxr-xr-x  3 root  root  4.0K Feb 19 21:33 target


# 3. Analyse descriptive:

In [ ]:
df.select("type_doc","Newsgroups","id").show()

+--------+--------------------+-----+
|type_doc|          Newsgroups|   id|
+--------+--------------------+-----+
| atheism|alt.atheism,talk....|53517|
| atheism|         alt.atheism|54221|
| atheism|talk.abortion,alt...|53089|
| atheism|         alt.atheism|53257|
| atheism|         alt.atheism|53758|
| atheism|         alt.atheism|54227|
| atheism|         alt.atheism|53473|
| atheism|         alt.atheism|53230|
| atheism|alt.atheism,talk....|53148|
| atheism|         alt.atheism|53537|
| atheism|rec.scouting,soc....|54234|
| atheism|         alt.atheism|53293|
| atheism|         alt.atheism|54219|
| atheism|         alt.atheism|53201|
| atheism|         alt.atheism|51197|
| atheism|         alt.atheism|51232|
| atheism|         alt.atheism|53594|
| atheism|         alt.atheism|53438|
| atheism|         alt.atheism|53158|
| atheism|talk.abortion,alt...|53091|
+--------+--------------------+-----+
only showing top 20 rows



## 3.a verifier qu'on a bien 2 catégories

In [ ]:
from pyspark.sql.functions import col, countDistinct
df.groupBy("type_doc").count().show()
df.groupBy("Organization").count().sort(col("count").desc()).show()


+--------+-----+
|type_doc|count|
+--------+-----+
|baseball|  994|
| atheism| 1000|
+--------+-----+

+--------------------+-----+
|        Organization|count|
+--------------------+-----+
|                null|  112|
|                 sgi|   70|
|California Instit...|   65|
|  Siemens-Nixdorf AG|   41|
|Mantis Consultant...|   40|
|Case Western Rese...|   39|
|Technical Univers...|   35|
|Okcforum Unix Use...|   32|
|University of Ill...|   32|
|Cookamunga Touris...|   25|
|Cornell Univ. CS ...|   25|
|Princeton University|   24|
|Netcom Online Com...|   23|
|Boston University...|   21|
|Tektronix Inc., B...|   20|
|   Allegheny College|   19|
|University of Ill...|   19|
|University of Den...|   18|
|University of Not...|   17|
|  Indiana University|   17|
+--------------------+-----+
only showing top 20 rows



## 3.b donner le nombre d'organisations differentes

In [ ]:
df.select(countDistinct("Organization")).show()

+----------------------------+
|count(DISTINCT Organization)|
+----------------------------+
|                         467|
+----------------------------+



## 3.c Autre analyses descriptive
### nombre d'element null par entete

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+-------+----+---------+----+----------+--------+------------+-------+--------+---+
|Subject|Path|corp_text|From|Newsgroups|type_doc|Organization|Summary|Keywords| id|
+-------+----+---------+----+----------+--------+------------+-------+--------+---+
|      0|   0|        0|   0|         0|       0|         112|   1960|    1937|  0|
+-------+----+---------+----+----------+--------+------------+-------+--------+---+



Fill les elements null de organisation(pour qu'ils ne soient plus considéré comme null)

In [ ]:
df = df.fillna( { 'Organization':""} )



In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+-------+----+---------+----+----------+--------+------------+-------+--------+---+
|Subject|Path|corp_text|From|Newsgroups|type_doc|Organization|Summary|Keywords| id|
+-------+----+---------+----+----------+--------+------------+-------+--------+---+
|      0|   0|        0|   0|         0|       0|           0|   1960|    1937|  0|
+-------+----+---------+----+----------+--------+------------+-------+--------+---+



### compter l'occurence de chaque sujet et ordonner par ordre décroissant

In [ ]:
df.groupBy("Subject").count().sort(col("count").desc()).show()


+--------------------+-----+
|             Subject|count|
+--------------------+-----+
|Re: After 2000 ye...|  130|
|Re: <Political At...|   46|
|Re: Yet more Rush...|   36|
|     Re: Jack Morris|   34|
|  Re: some thoughts.|   31|
|Re: Jewish Baseba...|   30|
|Re: free moral ag...|   27|
|Re: Rawlins debun...|   27|
|Re: Amusing athei...|   22|
|Re: Christian Mor...|   20|
|Re: Genocide is C...|   18|
|    Re: Albert Sabin|   18|
|Re: The Inimitabl...|   16|
|  Re: Young Catchers|   16|
|Re: YOU WILL ALL ...|   15|
|Re: HBP? BB? BIG-...|   14|
|Re: islamic autho...|   14|
|Re: And America's...|   14|
|Re: Notes on Jays...|   14|
|Re: Braves Pitchi...|   13|
+--------------------+-----+
only showing top 20 rows



#  4 / 5 / 6 Transformation du texte et clustering

Suppressions de colonne inutile lors du clustering (trop de null ou hors du sujet)

In [ ]:
from pyspark.sql.functions import concat, col, lit


columns_to_drop = ["Path","From","Summary","Keywords","id"]
df = df.drop(*columns_to_drop)

In [ ]:
df.show()

+--------------------+--------------------+--------------------+--------+--------------------+
|             Subject|           corp_text|          Newsgroups|type_doc|        Organization|
+--------------------+--------------------+--------------------+--------+--------------------+
|Re: After 2000 ye...|
In article <C5L1...|alt.atheism,talk....| atheism|Nottingham Univer...|
|Re: free moral ag...|Distribution: na
...|         alt.atheism| atheism|Case Western Rese...|
|Re: After 2000 ye...|Message-ID: <kmr4...|talk.abortion,alt...| atheism|Case Western Rese...|
|Re: Amusing athei...|Distribution: wor...|         alt.atheism| atheism|University of New...|
|Re: Yet more Rush...|
In article <1993...|         alt.atheism| atheism|Boston University...|
|Re: The Inimitabl...|Distribution: wor...|         alt.atheism| atheism|Case Western Rese...|
|Re: Christian Mor...|
In article <pww-...|         alt.atheism| atheism|Cookamunga Touris...|
|   Re: <<Pompous ass|
In article <1ql6...|       

transformation des caractéres en miniscule

In [ ]:
from pyspark.sql.functions import lower, col


df_final = df.select(lower(concat(col("corp_text"),lit('\n'),col("Subject"),lit('\n'),col("Organization"))).alias("FullText"))

df_final.show()


+--------------------+
|            FullText|
+--------------------+
|
in article <c5l1...|
|distribution: na
...|
|message-id: <kmr4...|
|distribution: wor...|
|
in article <1993...|
|distribution: wor...|
|
in article <pww-...|
|
in article <1ql6...|
|
in <1993apr15.07...|
|
in article <1993...|
|
[by default, fol...|
|message-id: <1r0s...|
|message-id: <kmr4...|
|message-id: <1qla...|
|message-id: <kmr4...|
|message-id: <1pm6...|
|
in article <1r0s...|
|
ed mccreary (edm...|
|
cmtan@iss.nus.sg...|
|message-id: <1qkn...|
+--------------------+
only showing top 20 rows



## 4.a + b + autres traitements:
 - découper les documents en liste à l'aide du tokenizer
 - suppression de stopords
 - créer une representation vectoriel à l'aide de hashing TF
 - regularisation L2norm

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, HashingTF, IDF, Normalizer

tokenizer = Tokenizer(inputCol="FullText", outputCol="words")
wordsData = tokenizer.transform(df_final)

In [ ]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
removedwordsData  = remover.transform(wordsData)
removedwordsData .select("filtered").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

haghing TF

In [ ]:
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1000)
featurizedData  = hashingTF.transform(removedwordsData)

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData  = idfModel.transform(featurizedData )


In [ ]:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
l2NormData = normalizer.transform(rescaledData)

Resultat final apres traitement

In [ ]:
l2NormData.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            FullText|               words|            filtered|         rawFeatures|            features|        normFeatures|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|
in article <c5l1...|[, in, article, <...|[, article, <c5l1...|(1000,[3,7,27,39,...|(1000,[3,7,27,39,...|(1000,[3,7,27,39,...|
|distribution: na
...|[distribution:, n...|[distribution:, n...|(1000,[2,39,47,73...|(1000,[2,39,47,73...|(1000,[2,39,47,73...|
|message-id: <kmr4...|[message-id:, <km...|[message-id:, <km...|(1000,[4,18,20,24...|(1000,[4,18,20,24...|(1000,[4,18,20,24...|
|distribution: wor...|[distribution:, w...|[distribution:, w...|(1000,[1,8,12,15,...|(1000,[1,8,12,15,...|(1000,[1,8,12,15,...|
|
in article <1993...|[, in, article, <...|[, article, <1993...|(1000,[10,12,13,1...|(1000,[10,12,13,1..

## 5 utilier l'algorithme kmean de spark

In [ ]:
%%time
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import pandas as pd 


kmeans = KMeans(featuresCol="normFeatures").setK(2).setSeed(1)
model = kmeans.fit(l2NormData)




CPU times: user 45.9 ms, sys: 7.27 ms, total: 53.2 ms
Wall time: 7.57 s


In [ ]:
# Make predictions
predictions = model.transform(l2NormData)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

centers = model.clusterCenters()


Silhouette with squared euclidean distance = -0.2795142616658651


In [ ]:
predictions.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|            FullText|               words|            filtered|         rawFeatures|            features|        normFeatures|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|
in article <c5l1...|[, in, article, <...|[, article, <c5l1...|(1000,[3,7,27,39,...|(1000,[3,7,27,39,...|(1000,[3,7,27,39,...|         0|
|distribution: na
...|[distribution:, n...|[distribution:, n...|(1000,[2,39,47,73...|(1000,[2,39,47,73...|(1000,[2,39,47,73...|         1|
|message-id: <kmr4...|[message-id:, <km...|[message-id:, <km...|(1000,[4,18,20,24...|(1000,[4,18,20,24...|(1000,[4,18,20,24...|         1|
|distribution: wor...|[distribution:, w...|[distribution:, w...|(1000,[1,8,12,15,...|(1000,[1,8,12,15,...|(1000,[1,8,12,15,...|         1|
|
in article <1993...|[, in

In [ ]:
from sklearn.metrics import normalized_mutual_info_score as nmi

kmeans_spark_prediction = np.array(predictions.select('prediction').collect()).reshape(-1)
kmeans_spark_prediction

array([0, 1, 1, ..., 1, 1, 0])

In [ ]:
original_labels = np.array(df.select('type_doc').collect())
original_labels = np.where(original_labels == "atheism", 0, 1).reshape(-1)

## 6 analyser les resultats + comparaison avec kmean de sklearn

On remarque que le NMI est assez faible ce qui est facilement explicable car kmeans n'est pas adapté à ce type de données.

In [ ]:
nmi(original_labels,kmeans_spark_prediction)

0.005954119237055068

transformation des données afin del les adapter à sklearn

In [ ]:
nomrfeatures_data = np.array(predictions.select('normFeatures').collect()).reshape((1994,1000))

In [ ]:
nomrfeatures_data.shape

(1994, 1000)

In [ ]:
features_sklearn = np.array(predictions.select('normFeatures').collect()).shape

Lancement de l'algorithme kmean en utilisant le package sklearn

In [ ]:
%%time
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(nomrfeatures_data)

CPU times: user 1.74 s, sys: 1.04 s, total: 2.79 s
Wall time: 1.61 s


In [ ]:
kmeans.labels_

array([1, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
nmi(original_labels,kmeans.labels_)

0.02242000211585393

### Remarque
on remarque que dans ce cas ci, sklearn est plus rapide à executer que spark, ceci s'explique car le volume des données n'est pas assez important pour justifier l'utilisation de spark.

Aussi sklearn donne une nmi superieur bien que tout aussi mauvaise pour les méme raisons que celle cité ci-dessus.

# 7. kmeans unidimensionnel

In [ ]:
cluster_ids = sc.parallelize([5,4,3,2,2,2,2,1,2,2,2,3,3])
point = sc.parallelize([x for x in range(0,500)])


## 7.a définir la fonction compute centroid

In [ ]:
from numpy import random

def compute_centroids(points, cluster_ids):

  sum_by_cluster_id = cluster_ids.zip(points).reduceByKey(lambda x,y: x+y)
  count_by_cluster_id = cluster_ids.zip(points).mapValues(lambda x:1).reduceByKey(lambda x,y: x+y)
  couple = sum_by_cluster_id.join(count_by_cluster_id).mapValues(lambda x: x[0] / x[1]).sortByKey()

  return couple



## 7.b éfinir la fonction assign_cluster

7.b.ii dans ce cas ci nous supposons  qu'on dispose d'assez de mémoire pour ne pas saturer le driver program.

In [ ]:

def squared_distances(point_x, vecteur_moy):

 return np.sqrt((np.array(vecteur_moy).reshape(len(vecteur_moy),1) - point_x)**2).reshape(-1)

def assign_clusters(points, centroids):

  vecteur_moy = centroids.map(lambda x: x[1]).collect() #  dans ce cas ci nous supposons  qu'on dispose d'assez de mémoire pour ne pas saturer le driver program.
  return points.map(lambda x: np.argmin(squared_distances(x,vecteur_moy)))


   


## 7.c implémenter l'etape d'initialisation et itération

In [ ]:

def kmeans_unidim(points,k=2,max_iter=100):

#  vec =  np.random.rand(k)
  vec = points.takeSample(False, k)
  centroids = sc.parallelize(zip(range(k),vec))

  for i in range(0,max_iter):
    cluster_ids = assign_clusters(points, centroids)
    centroids = compute_centroids(points, cluster_ids)

  cluster_ids = assign_clusters(points, centroids)


  return centroids, cluster_ids



In [ ]:
%%time
centroids, cluster_ids = kmeans_unidim(point,2,10)

CPU times: user 840 ms, sys: 73.1 ms, total: 913 ms
Wall time: 11.8 s


In [ ]:
print("centroids")
print(centroids.collect())
print("cluster_ids")
print(cluster_ids.collect())

centroids
[(0, 374.0), (1, 124.0)]
cluster_ids
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# 8. Kmeans multidimensionnel

modification des fonctions compute_centroid et squared_distances afin de l'adapter à des calcules vectoriel via numpy (np.add, np.divide ...)


In [ ]:
from numpy import random
def compute_centroids(points, cluster_ids):

  sum_by_cluster_id = cluster_ids.zip(points).reduceByKey(lambda x,y: np.add(x,y).tolist())
  count_by_cluster_id = cluster_ids.zip(points).mapValues(lambda x:1).reduceByKey(lambda x,y: x+y)
  couple = sum_by_cluster_id.join(count_by_cluster_id).mapValues(lambda x: np.divide(x[0], x[1]).tolist()).sortByKey()

  return couple


def squared_distances(point_x, vecteur_moy):

 return np.sqrt(np.sum((np.array(vecteur_moy) - point_x)**2,axis=1))

def assign_clusters(points, centroids):

  vecteur_moy = centroids.map(lambda x: x[1]).collect() #supposition qu'on est entrain de faire est qu'on dispose assez de memeoire 
  a = points.map(lambda x: np.argmin(squared_distances(x,vecteur_moy)))
  return a



def kmeans_multidim(points,k=2,max_iter=100):

  vec = points.takeSample(False, k)
  centroids = sc.parallelize(zip(range(0,k),vec))

  for i in range(0,max_iter):
    cluster_ids = assign_clusters(points, centroids)
    centroids = compute_centroids(points, cluster_ids)

  cluster_ids = assign_clusters(points, centroids)


  return centroids, cluster_ids



   



In [ ]:
%%time
points = sc.parallelize([[1,2], [4,5], [7,8], [11,12]])
centroids, cluster_ids = kmeans_multidim(points, 2, 5)

CPU times: user 456 ms, sys: 44 ms, total: 500 ms
Wall time: 5.85 s


In [ ]:
print("centroids")
print(centroids.collect())
print("cluster_ids")
print(cluster_ids.collect())

centroids
[(0, [9.0, 10.0]), (1, [2.5, 3.5])]
cluster_ids
[1, 1, 0, 0]


# 9. Spherical Kmean

Modification de la fonction de distance afin d'utiliser la distance du cosine dans assign_cluster, nous utilisons pour cela la fonction cosine_similarity de sklearn.
Nous ne voulons donc plus récupérer l'argument minimum dans assign_cluster lors de l'assignation d'un individu à un centre (et donc cluster) mais l'argument maximum (le centre le plus proche et celui disposant de la plus grand cosine similarity)

In [ ]:
from numpy import random
from sklearn.metrics.pairwise import cosine_similarity

def compute_centroids(points, cluster_ids):

  sum_by_cluster_id = cluster_ids.zip(points).reduceByKey(lambda x,y: np.add(x,y).tolist())
  count_by_cluster_id = cluster_ids.zip(points).mapValues(lambda x:1).reduceByKey(lambda x,y: x+y)
  couple = sum_by_cluster_id.join(count_by_cluster_id).mapValues(lambda x: np.divide(x[0], x[1]).tolist()).mapValues(lambda x: np.divide(x, np.linalg.norm(x)).tolist()).sortByKey()

  return couple

def cosine_distance(point_x, vecteur_moy):

 return cosine_similarity(np.array(vecteur_moy), np.array([point_x])).reshape(-1)

def assign_clusters(points, centroids):

  vecteur_moy = centroids.map(lambda x: x[1]).collect() #supposition qu'on est entrain de faire est qu'on dispose assez de memeoire 
  a = points.map(lambda x: np.argmax(cosine_distance(x,vecteur_moy)))
  return a



def skmeans_multidim(points,k=2,max_iter=100):

  vec = points.takeSample(False, k)
  centroids = sc.parallelize(zip(range(0,k),vec))

  for i in range(0,max_iter):
    cluster_ids = assign_clusters(points, centroids)
    centroids = compute_centroids(points, cluster_ids)

  cluster_ids = assign_clusters(points, centroids)


  return centroids, cluster_ids


# 10. analyser les resultat et comparer au package coclust

In [ ]:
data = nomrfeatures_data[~np.all(nomrfeatures_data == 0, axis=1)]
data= np.delete(data,np.where(~data.any(axis=0))[0], axis=1)


In [ ]:
points = sc.parallelize(data)


In [ ]:
%%time
centroids, cluster_ids = skmeans_multidim(points, 2, 100)

CPU times: user 10.9 s, sys: 718 ms, total: 11.6 s
Wall time: 5min 16s


In [ ]:
centroids.take(10)

[(0,
  [0.008756995418851252,
   0.00207862977462898,
   0.008538098250457796,
   0.0042402571605317995,
   0.008121422893886378,
   0.011553300975241976,
   0.008176171495657622,
   0.00661600165191711,
   0.014252393151618697,
   0.005665462480497547,
   0.015579178748943484,
   0.006715207864802862,
   0.007912467201543942,
   0.014104756978691447,
   0.0048877040992437795,
   0.013462093064177645,
   0.003489542062895842,
   0.009539472757894348,
   0.007792167956933527,
   0.009493535126753116,
   0.029434792487041894,
   0.009025744920883799,
   0.008344719368026525,
   0.03924100209364731,
   0.013099959887829806,
   0.008752970891396712,
   0.005414427152908825,
   0.013360981474793478,
   0.004574665409049534,
   0.004458180932085855,
   0.00484176789621574,
   0.005220968876374344,
   0.0030191330143505162,
   0.012137616257681609,
   0.008115550007122958,
   0.014541341584104714,
   0.009705342665961259,
   0.020086038103663537,
   0.012332345773887646,
   0.0143164597474546

In [ ]:
cluster_ids.take(10)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Skmean avec coclust

In [ ]:
%%time

from coclust.clustering import SphericalKmeans
import scipy as sp
from scipy import sparse

skmean_model = SphericalKmeans( max_iter=100, n_init=1, random_state=123, weighting=False)
skmean_model.fit(sparse.csr_matrix(data))

 == New init == 
iteration: 0
590.3333876247439
iteration: 1
602.385220436727
iteration: 2
608.065900254631
iteration: 3
611.4016323711354
iteration: 4
614.7181148569493
iteration: 5
618.7072412660007
iteration: 6
622.0005947470472
iteration: 7
627.8337578434423
iteration: 8
637.9775150303957
iteration: 9
639.0680004803286
iteration: 10
639.0806199589309
iteration: 11
CPU times: user 1.61 s, sys: 17.5 ms, total: 1.62 s
Wall time: 1.62 s


In [ ]:
skmean_model.labels_

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,


NMI de notre implémentation

In [ ]:
nmi(original_labels,cluster_ids.collect())

0.017111880127824775

NMI de l'implémentation coclust

In [ ]:
nmi(original_labels,skmean_model.labels_)

0.017111880127824775

### Remarque :
On peut remarquer qu'en terme de qualité des resutlats sur nos données textuel precedentes en utilisant la NMI, les deux implémentations ont des resultats similaire
.

Cependant Coclust est beaucoup plus rapide.

# 11. MLIB Classification non supervisé

## GMM model

In [ ]:

sample_data = sc.parallelize([[0.0, 0.0, 0.0],
[0.1, 0.1, 0.1],
[0.2, 0.2, 0.2],
[9.0, 9.0, 9.0],
[9.1, 9.1, 9.1],
[9.2, 9.2, 9.2]])

In [ ]:
from numpy import array

from pyspark.mllib.clustering import GaussianMixture, GaussianMixtureModel

# Build the model (cluster the data)
gmm = GaussianMixture.train(sample_data, 2)

# output parameters of model
for i in range(2):
    print("weight = ", gmm.weights[i], "mu = ", gmm.gaussians[i].mu,
          "sigma = ", gmm.gaussians[i].sigma.toArray())

print("GMM predictions on sample data") 
print(gmm.predict(sample_data).collect())

weight =  0.4988113094096504 mu =  [4.551406083942625,4.551406083942625,4.551406083942625] sigma =  [[20.2543053 20.2543053 20.2543053]
 [20.2543053 20.2543053 20.2543053]
 [20.2543053 20.2543053 20.2543053]]
weight =  0.5011886905903497 mu =  [4.648363411531433,4.648363411531433,4.648363411531433] sigma =  [[20.25432765 20.25432765 20.25432765]
 [20.25432765 20.25432765 20.25432765]
 [20.25432765 20.25432765 20.25432765]]
predictions on sample data
[0, 0, 0, 1, 1, 1]


In [ ]:
from numpy import array
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel

# Build the model (cluster the data)
kmean_spark = KMeans.train(sample_data, 2, maxIterations=10, initializationMode="random")

print("GMM predictions on sample data")
print(kmean_spark.predict(sample_data).collect())

GMM predictions on sample data
[1, 1, 1, 0, 0, 0]


# 12. MLIB classification supervisé

In [ ]:
from pyspark.mllib.util import MLUtils


training = spark.read.format("libsvm").load('/content/sample_libsvm_data.txt')


## Logistic regression

In [ ]:
from pyspark.ml.classification import LogisticRegression


lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

Coefficients: (692,[244,263,272,300,301,328,350,351,378,379,405,406,407,428,433,434,455,456,461,462,483,484,489,490,496,511,512,517,539,540,568],[-7.353983524188197e-05,-9.102738505589466e-05,-0.00019467430546904298,-0.00020300642473486668,-3.1476183314863995e-05,-6.842977602660743e-05,1.5883626898239883e-05,1.4023497091372047e-05,0.00035432047524968605,0.00011443272898171087,0.00010016712383666666,0.0006014109303795481,0.0002840248179122762,-0.00011541084736508837,0.000385996886312906,0.000635019557424107,-0.00011506412384575676,-0.00015271865864986808,0.0002804933808994214,0.0006070117471191634,-0.0002008459663247437,-0.0001421075579290126,0.0002739010341160883,0.00027730456244968115,-9.838027027269332e-05,-0.0003808522443517704,-0.00025315198008555033,0.00027747714770754307,-0.0002443619763919199,-0.0015394744687597765,-0.00023073328411331293])
Intercept: 0.22456315961250325
Multinomial coefficients: 2 X 692 CSRMatrix
(0,244) 0.0
(0,263) 0.0001
(0,272) 0.0001
(0,300) 0.0001
(0,350) 

## Random forest

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Index labels, adding metadata to the label column.
data = training
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[121,122,123...|
|           0.0|  0.0|(692,[123,124,125...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[126,127,128...|
|           0.0|  0.0|(692,[126,127,128...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0
RandomForestClassificationModel: uid=RandomForestClassifier_cc6727bd9514, numTrees=10, numClasses=2, numFeatures=692
